In [1]:
import numpy as np, os
import pandas as pd
import matplotlib.pyplot as plt
import os

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

import tensorflow as tf
import tensorflow.keras.backend as K
import tokenizers
from transformers import ElectraTokenizer, TFElectraModel

from collections import Counter

import warnings
warnings.filterwarnings("ignore")

2022-08-28 16:33:37.361988: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-28 16:33:37.363178: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-28 16:33:37.363871: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-28 16:33:37.366270: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [2]:
import os
os.listdir('/kaggle/input')

['machinegeneratedtestdatav2', 'testversion3', 'machinegenv3', 'machinegenv2']

In [3]:

train_data = pd.read_csv('../input/machinegenv3/train_human_machine.csv').dropna()['review'].to_list()
test_data = pd.read_csv('/kaggle/input/testversion3/train_auto_gen.csv').dropna()['review'].to_list()
valid_data = pd.read_csv('/kaggle/input/testversion3/train_auto_gen.csv').dropna()['review'].to_list()
print(len(train_data) , len(test_data))
train_label = pd.read_csv('../input/machinegenv3/train_human_machine.csv').dropna()['auto_gen_label']
test_label = pd.read_csv('/kaggle/input/testversion3/train_auto_gen.csv').dropna()['auto_gen_label']
valid_label = pd.read_csv('/kaggle/input/testversion3/train_auto_gen.csv').dropna()['auto_gen_label']
    
print(len(train_label) , len(test_label))    
    
    
    

2555 179
2555 179


In [4]:
print(type(train_label))

<class 'pandas.core.series.Series'>


In [5]:
MODEL_NAME = 'google/electra-base-discriminator'
MAX_LEN = 512
ARTIFACTS_PATH = '../artifacts/'

BATCH_SIZE = 8
EPOCHS = 15

if not os.path.exists(ARTIFACTS_PATH):
    os.makedirs(ARTIFACTS_PATH)

In [6]:
def electra_encode(texts, tokenizer):
    ct = len(texts)
    input_ids = np.ones((ct, MAX_LEN), dtype='int32')
    attention_mask = np.zeros((ct, MAX_LEN), dtype='int32')
    token_type_ids = np.zeros((ct, MAX_LEN), dtype='int32') # Not used in text classification

    for k, text in enumerate(texts):
        # Tokenize
        tok_text = tokenizer.tokenize(text)
        
        # Truncate and convert tokens to numerical IDs
        enc_text = tokenizer.convert_tokens_to_ids(tok_text[:(MAX_LEN-2)])
        
        input_length = len(enc_text) + 2
        input_length = input_length if input_length < MAX_LEN else MAX_LEN
        
        # Add tokens [CLS] and [SEP] at the beginning and the end
        input_ids[k,:input_length] = np.asarray([0] + enc_text + [2], dtype='int32')
        
        # Set to 1s in the attention input
        attention_mask[k,:input_length] = 1

    return {
        'input_word_ids': input_ids,
        'input_mask': attention_mask,
        'input_type_ids': token_type_ids
    }

In [7]:
tokenizer = ElectraTokenizer.from_pretrained(MODEL_NAME)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/27.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/666 [00:00<?, ?B/s]

In [8]:
train_data = electra_encode(train_data, tokenizer)
valid_data = electra_encode(valid_data, tokenizer)
test_data = electra_encode(test_data, tokenizer)

In [9]:
y_data_train = pd.read_csv('../input/machinegenv3/train_human_machine.csv').dropna()['auto_gen_label'].to_numpy().reshape(-1)

In [10]:
category_to_id = {}
category_to_name = {}

for index, c in enumerate(y_data_train):
    if c in category_to_id:
        category_id = category_to_id[c]
    else:
        category_id = len(category_to_id)
        category_to_id[c] = category_id
        category_to_name[category_id] = c
    
    y_data_train[index] = category_id

# Display dictionary
category_to_name

{0: 'Positive', 1: 'Negative'}

In [11]:
print(y_data_train)

[0 1 0 ... 0 1 1]


In [12]:
y_data_test = pd.read_csv('/kaggle/input/testversion3/train_auto_gen.csv').dropna()['auto_gen_label'].to_numpy().reshape(-1)

In [13]:
category_to_id = {}
category_to_name = {}

for index, c in enumerate(y_data_test):
    if c in category_to_id:
        category_id = category_to_id[c]
    else:
        category_id = len(category_to_id)
        category_to_id[c] = category_id
        category_to_name[category_id] = c
    
    y_data_test[index] = category_id

# Display dictionary
category_to_name

{0: 'Positive', 1: 'Negative'}

In [14]:
print(y_data_test)

[0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 1 0 1 0 1
 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 1 0 1 0 1 0 1 0 1 1 0 1 1 1 1 0 1 0 1 1 1
 0 1 0 1 0 1 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 1 0 1 1 0 1 0 1 0 1 1 0 1 0 1 1
 0 1 0 1 0 1 0 1 0 1 0 1 1 0 1 0 1 0 1 1 0 1 0 1 1 0 1 0 1 0 1 0 1 0 1 1 0
 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 1 0 1 0 1 0 1 1 0 1 1 1 0 1 0 1]


In [15]:
y_data_test = np.asarray(y_data_test, dtype='int32')
y_data_train = np.asarray(y_data_train, dtype='int32')

In [16]:
print(valid_data)

{'input_word_ids': array([[   0, 6423, 2775, ...,    1,    1,    1],
       [   0, 6423, 2775, ...,    1,    1,    1],
       [   0, 1041, 1012, ...,    1,    1,    1],
       ...,
       [   0, 6804, 1011, ...,    1,    1,    1],
       [   0, 6097, 1010, ...,    1,    1,    1],
       [   0, 3532, 3737, ...,    1,    1,    1]], dtype=int32), 'input_mask': array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]], dtype=int32), 'input_type_ids': array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)}


In [17]:
def build_model(n_categories):

    input_word_ids = tf.keras.Input(shape=(MAX_LEN,), dtype=tf.int32, name='input_word_ids')
    input_mask = tf.keras.Input(shape=(MAX_LEN,), dtype=tf.int32, name='input_mask')
    input_type_ids = tf.keras.Input(shape=(MAX_LEN,), dtype=tf.int32, name='input_type_ids')

    # Import RoBERTa model from HuggingFace
    electra_model = TFElectraModel.from_pretrained(MODEL_NAME)
    electra_model.trainable = False
    x = electra_model(input_word_ids, attention_mask=input_mask, token_type_ids=input_type_ids)

    # Huggingface transformers have multiple outputs, embeddings are the first one,
    # so let's slice out the first position

    x = x[0]

    x = tf.keras.layers.Dropout(0.33)(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(128, activation='relu')(x)
    x = tf.keras.layers.Dense(1, activation='sigmoid')(x)

    model = tf.keras.Model(inputs=[input_word_ids, input_mask, input_type_ids], outputs=x)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(lr=1e-5),
        loss='binary_crossentropy',
        metrics=['accuracy'])

    return model

In [18]:
model = build_model(2)
model.summary()

Downloading:   0%|          | 0.00/418M [00:00<?, ?B/s]

Some layers from the model checkpoint at google/electra-base-discriminator were not used when initializing TFElectraModel: ['discriminator_predictions']
- This IS expected if you are initializing TFElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFElectraModel were initialized from the model checkpoint at google/electra-base-discriminator.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFElectraModel for predictions without further training.


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 512)]        0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 512)]        0                                            
__________________________________________________________________________________________________
input_type_ids (InputLayer)     [(None, 512)]        0                                            
__________________________________________________________________________________________________
tf_electra_model (TFElectraMode TFBaseModelOutputWit 108891648   input_word_ids[0][0]             
                                                                 input_mask[0][0]             

In [19]:
class config:
    MODEL_SAVE_DIR = './checkpoint'

cfg = config()
cfg

In [20]:
from tensorflow.keras import callbacks
def get_callback_list():
    #to do : add lr schedular, tensorboad log
    early_stop = callbacks.EarlyStopping(monitor="accuracy", mode="max",restore_best_weights=True, patience=4)
    checkpoint = callbacks.ModelCheckpoint(filepath=os.path.join(cfg.MODEL_SAVE_DIR, "{epoch:03d}-{val_accuracy:.3f}.h5"),
                                save_weights_only=True,
                                monitor='val_accuracy',
                                model='max',
                                save_best_only=True,
                                )
    csv_logs = callbacks.CSVLogger(os.path.join(cfg.MODEL_SAVE_DIR, "logs.csv"))
    reduce_lr = callbacks.ReduceLROnPlateau(factor=0.5, monitor="val_accuracy", mode="max", patience=3)
    tb = callbacks.TensorBoard(log_dir= os.path.join(cfg.MODEL_SAVE_DIR, "logs"))

    return [checkpoint, tb]

In [21]:
print('Training...')
history = model.fit(train_data,
                    y_data_train,
                    epochs=10,
                    batch_size=16,
                    verbose=1,
                    callbacks = get_callback_list(),
                    validation_data=(valid_data, y_data_test))

Training...


2022-08-28 16:34:22.204678: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2022-08-28 16:34:22.204722: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2022-08-28 16:34:22.206997: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1614] Profiler found 1 GPUs
2022-08-28 16:34:22.723349: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2022-08-28 16:34:22.723566: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1748] CUPTI activity buffer flushed
2022-08-28 16:34:22.944022: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/10
  1/160 [..............................] - ETA: 29:57 - loss: 0.9506 - accuracy: 0.5000

2022-08-28 16:34:34.660214: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2022-08-28 16:34:34.660271: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.


  2/160 [..............................] - ETA: 2:59 - loss: 0.8853 - accuracy: 0.5625 

2022-08-28 16:34:35.446300: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2022-08-28 16:34:35.448378: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1748] CUPTI activity buffer flushed
2022-08-28 16:34:35.602738: I tensorflow/core/profiler/internal/gpu/cupti_collector.cc:673]  GpuTracer has collected 1248 callback api events and 1245 activity events. 
2022-08-28 16:34:35.644075: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.


160/160 [==============================] - 88s 484ms/step - loss: 0.3659 - accuracy: 0.8301 - val_loss: 0.2938 - val_accuracy: 0.8939
Epoch 2/10
160/160 [==============================] - 74s 462ms/step - loss: 0.2095 - accuracy: 0.9123 - val_loss: 0.1586 - val_accuracy: 0.9665
Epoch 3/10
160/160 [==============================] - 74s 462ms/step - loss: 0.1687 - accuracy: 0.9358 - val_loss: 0.1402 - val_accuracy: 0.9665
Epoch 4/10
160/160 [==============================] - 74s 460ms/step - loss: 0.1210 - accuracy: 0.9534 - val_loss: 0.1425 - val_accuracy: 0.9721
Epoch 5/10
160/160 [==============================] - 74s 461ms/step - loss: 0.1107 - accuracy: 0.9581 - val_loss: 0.1503 - val_accuracy: 0.9665
Epoch 6/10
160/160 [==============================] - 74s 460ms/step - loss: 0.1069 - accuracy: 0.9613 - val_loss: 0.1619 - val_accuracy: 0.9721
Epoch 7/10
160/160 [==============================] - 73s 454ms/step - loss: 0.1021 - accuracy: 0.9648 - val_loss: 0.1594 - val_accuracy: 0.9

In [22]:
print(valid_label)

0      Positive
1      Negative
2      Positive
3      Negative
4      Positive
         ...   
195    Negative
196    Positive
197    Negative
198    Positive
199    Negative
Name: auto_gen_label, Length: 179, dtype: object
